In [45]:
import whenever
whenever.Instant.now().format_rfc3339().replace(":", "-")

'2024-10-09 10-35-16.2379549Z'

In [55]:
now = whenever.Instant.now()
now.py_datetime().strftime("%Y-%m-%d %H-%M-%S")

'2024-10-09 10-40-50'

In [49]:
now = whenever.Instant.now().to_system_tz().to_fixed_offset()
print(now.format_rfc3339())
print(now.format_common_iso())
# now.to_system_tz().to_fixed_offset().format_rfc3339()

2024-10-09 12:37:11.9798553+02:00
2024-10-09T12:37:11.9798553+02:00


In [1]:
import uwacan
import plotly.graph_objects as go
import plotly
import plotly.colors as pco
import plotly.subplots
import xarray as xr
import numpy as np

plotly.io.templates.default = 'plotly_white'

from pathlib import Path

# from importlib import reload
# reload(uwacan.positional.implementations)
# reload(uwacan.positional._wrappers)
# reload(uwacan.positional)
# reload(uwacan.analysis)
# reload(uwacan.recordings)

data_path = Path(r'D:\Tern Island Vinga')

# Position track

In [ ]:
position_track = uwacan.Track.read_nmea_gps(r'C:\Users\carl4189\OneDrive - IVL Svenska Miljöinstitutet AB\Dokument\Projects\PUB\GPS Kongsberg.txt')
position_track["depth"] = 5
fig = position_track.make_figure()
fig.add_trace(position_track.plot(hover_data={"speed": True}))

In [3]:
transit_tracks = [
    position_track.subwindow(start='2023-08-30 12:18:00z', stop='2023-08-30 12:23:00z'),
    position_track.subwindow(start='2023-08-30 12:36:00z', stop='2023-08-30 12:41:00z'),
    position_track.subwindow(start='2023-08-30 12:58:00z', stop='2023-08-30 13:02:00z'),
    position_track.subwindow(start='2023-08-30 13:29:00z', stop='2023-08-30 13:34:00z'),
    position_track.subwindow(start='2023-08-30 13:58:00z', stop='2023-08-30 14:03:00z'),
    position_track.subwindow(start='2023-08-30 14:30:00z', stop='2023-08-30 14:36:00z'),
    position_track.subwindow(start='2023-08-30 15:02:00z', stop='2023-08-30 15:06:00z'),
]

# Sensor positions
From the sea chart and some mild interpolation, it seems to be a reasonable assumption that the water depth for all sensors is 46 meters.
We have the following measurement setup:

Sensor | Distance from track | Height above bottom | Position
-------|---------------------|---------------------|---------
Sylence 1 | 0 | 2 | 57.62300, 11.5775
Soundtrap 1 | 40 | 2 | 57.62325, 11.57790
Soundtrap 2 | 70 | 2 | 57.62350, 11.57827
Soundtrap 3 | 70 | 20 | 57.62350, 11.57827
Colmar 1 (155) | 147 | 2 | 57, 37.3070, 11, 34.4888
Colmar 2 (156) | 147 | 12 | 57, 37.3070, 11, 34.4888
Colmar 3 (157) (No signal) | 147 | 22 | 57, 37.3070, 11, 34.4888
Sylence 2 | 200 | 2 | 57, 37.5036, 11, 34.7163
Soundtrap 4 (Foi) | 200 | 15 | 57, 37.5036, 11, 34.7163

In [4]:
auto_sensors = (
    uwacan.sensor('SoundTrap 1', depth=46 - 2, sensitivity=-176.6, position= (57.62325, 11.57790))  # SN 5305
    & uwacan.sensor('SoundTrap 2', depth=46 - 2, sensitivity=-176.4, position= (57.62350, 11.57827))  # SN 5300
    & uwacan.sensor('SoundTrap 3', depth=46 - 20, sensitivity=-174.8, position=(57.62350, 11.57827))  # SN 5485
    & uwacan.sensor('SoundTrap 4', depth=46 - 15, sensitivity=-174.0, position=(57, 37.5036, 11, 34.7163))
    & uwacan.sensor('Sylence 1', depth=46 - 2, sensitivity=-179.2, position= (57.62300, 11.5775))
    & uwacan.sensor('Sylence 2', depth=46 - 2, sensitivity=-178.7, position= (57, 37.5036, 11, 34.7163))
)
cable_sensors = (
    uwacan.sensor('Colmar 1', depth=46 - 2, sensitivity=-164.09, position=(57, 37.3070, 11, 34.4888))
    & uwacan.sensor('Colmar 2', depth=46 - 12, sensitivity=-164.24, position=(57, 37.3070, 11, 34.4888))
)
all_sensors = auto_sensors & cable_sensors

# Recorders

In [5]:
soundtrap_1 = uwacan.recordings.SoundTrap.read_folder(
    data_path / 'Autonomous Recorders' / 'IVL SoundTrap 1',
    sensor=auto_sensors.sensors['SoundTrap 1'],
    time_compensation=uwacan.recordings.TimeCompensation('2023-08-30 15:04:17.3z', '2023-08-30 17:04:04.1z'),
    # time_compensation=7200,  # Yup, time is in local...
)
soundtrap_2 = uwacan.recordings.SoundTrap.read_folder(
    data_path / 'Autonomous Recorders' / 'IVL SoundTrap 2',
    sensor=auto_sensors.sensors['SoundTrap 2'],
    time_compensation=uwacan.recordings.TimeCompensation('2023-08-30 15:04:17.3z', '2023-08-30 17:04:08.1z'),
    # time_compensation=7200,  # Yup, time is in local...
)
soundtrap_3 = uwacan.recordings.SoundTrap.read_folder(
    data_path / 'Autonomous Recorders' / 'IVL SoundTrap 3',
    sensor=auto_sensors.sensors['SoundTrap 3'],
    time_compensation=uwacan.recordings.TimeCompensation('2023-08-30 15:04:17.3z', '2023-08-30 17:04:08.5z'),
    # time_compensation=7200,  # Yup, time is in local...
)
soundtrap_foi = uwacan.recordings.SoundTrap.read_folder(
    data_path / 'Autonomous Recorders' / 'FOI SoundTrap',
    sensor=auto_sensors.sensors['SoundTrap 4'],
    time_compensation=uwacan.recordings.TimeCompensation('2023-08-30 15:04:17.3z', '2023-08-30 17:04:15.2z'),
    # time_compensation=7200,  # Yup, time is in local...
)
sylence_1 = uwacan.recordings.SylenceLP.read_folder(
    data_path / 'Autonomous Recorders' / 'IVL Sylence 1',
    sensor=auto_sensors.sensors['Sylence 1'],
    time_compensation=uwacan.recordings.TimeCompensation('2023-08-30 15:04:17.3z', '2023-08-30 15:04:17.3z'),
)
sylence_2 = uwacan.recordings.SylenceLP.read_folder(
    data_path / 'Autonomous Recorders' / 'IVL Sylence 2',
    sensor=auto_sensors.sensors['Sylence 2'],
    time_compensation=uwacan.recordings.TimeCompensation('2023-08-30 15:04:17.3z', '2023-08-30 15:04:18.4z'),
)

In [6]:
fireface_recordings = uwacan.recordings.MultichannelAudioInterfaceRecording.read_folder(
    folder=data_path / 'Manual Recordings' / 'Fireface',
    # start_time_parser='YYYY-MM-DD_HH-mm-ss',
    start_time_parser='%Y-%m-%d_%H-%M-%S',
    sensor=cable_sensors,
    # adc_range=10**(19/20) * 2**0.5 * 0.6**0.5,
    adc_range=uwacan.recordings.dBx_to_peak_volts("19 dBu"),
    channel={'Colmar 1': 0, 'Colmar 2': 1},
    glob_pattern='*.wav'
)

fireface_transits = [
    uwacan.Transit(fireface_recordings.select_file_time(
        track.closest_point(fireface_recordings.sensor)["time"]
    ), track)
    for track in transit_tracks
]

## Plotting single passages

In [ ]:
transit = transit_tracks[0]
recording = fireface_recordings
# recording = soundtrap_1
cpa = transit.closest_point(recording.sensor)
time_data = recording.subwindow(center=cpa.time, duration=120).time_data()
if "sensor" in time_data.dims:
    time_data = time_data.sel(sensor="Colmar 1")

spec = uwacan.spectral.Spectrogram(
    time_data,
    bands_per_decade=100,
    frame_step=0.5,
    frame_duration=1,
    # hybrid_resolution=1,
    hybrid_resolution=True,
    min_frequency=5,
    max_frequency=40_000,
)
fig = spec.make_figure()
fig.add_trace(spec.plot())
fig.show()
# go.Figure(
#     go.Heatmap(
#         x=spec.time,
#         y=spec.frequency,
#         z=uwacan.dB(spec, power=True)
#     ),
#     {
#         "yaxis": {"type": "log"}
#     }
# ).show()


# Propagation model

In [8]:
propagation_model = uwacan.propagation.SeabedCriticalAngle(
    water_depth=46,
    substrate_compressional_speed=1507,
    speed_of_sound=1503,
)

In [ ]:
frequency = np.geomspace(10, 10_000)
go.Figure(
    [
        go.Scatter(
            x=frequency,
            y=uwacan.dB(propagation_model.power_propagation(distance=50, source_depth=5, receiver_depth=40, frequency=frequency)),
            name='50 m'
        ),
        go.Scatter(
            x=frequency,
            y=uwacan.dB(propagation_model.power_propagation(distance=100, source_depth=5, receiver_depth=40, frequency=frequency)),
            name='100 m'
        ),
        go.Scatter(
            x=frequency,
            y=uwacan.dB(propagation_model.power_propagation(distance=200, source_depth=5, receiver_depth=40, frequency=frequency)),
            name='200 m'
        ),
    ],
    {
        'xaxis': {'type': 'log', 'title': 'Frequency'},
        'yaxis': {'title': 'PL'},
    }
).show()

distance = np.geomspace(5, 300)
go.Figure(
    [
        go.Scatter(
            x=distance,
            y=uwacan.dB(propagation_model.power_propagation(distance=distance, source_depth=5, receiver_depth=40, frequency=10)),
            name='10 Hz',
        ),
        go.Scatter(
            x=distance,
            y=uwacan.dB(propagation_model.power_propagation(distance=distance, source_depth=5, receiver_depth=40, frequency=100)),
            name='100 Hz',
        ),
        go.Scatter(
            x=distance,
            y=uwacan.dB(propagation_model.power_propagation(distance=distance, source_depth=5, receiver_depth=40, frequency=1000)),
            name='1000 Hz',
        ),
    ],
    {
        'xaxis': {'type': 'log', 'title': 'Range'},
        'yaxis': {'title': 'PL'},
    }
).show()


# Analysis settings

In [10]:
filterbank = uwacan.spectral.Spectrogram(
    # window_duration=5,
    frame_overlap=0.5,
    bands_per_decade=100,
    hybrid_resolution=0.2,
    min_frequency=5,
    max_frequency=40e3,
)

In [ ]:
mean_background.sel(frequency=1e3)

In [11]:
recording = fireface_recordings.select_file_time("2023-08-30 12:32:57z")
time_data = recording.time_data()
# mean_background = filterbank(time_data).mean("time")
mean_background = uwacan.spectral.Spectrogram(
    time_data,
    frame_step=2,
    bands_per_decade=10,
    min_frequency=20,
    max_frequency=20_000,
).mean("time")
background_noise = uwacan.background.Background(mean_background)

In [14]:
ship_level = uwacan.analysis.ShipLevel.analyze_transits(
    *fireface_transits,
    filterbank=filterbank,
    propagation_model=propagation_model,
    background_noise=background_noise,
    transit_min_angle=30,
    transit_min_duration=30,
)

In [ ]:
# source_level = ship_level.sel(transit=slice(4, 7)).mean(["segment", "transit", "sensor"]).source_level
source_level = ship_level.sel(transit=slice(4, 7)).power_average(["sensor", "segment"]).level_average("transit").source_level
source_level = uwacan.analysis.convert_to_radiated_noise(source_level, 5, mode="iso", power=False)
go.Figure(go.Scatter(x=source_level.frequency, y=source_level)).update_xaxes(type="log")

In [ ]:
fig = go.Figure(layout=dict(
    xaxis=dict(
        type="log"
    )
))
for transit, data in ship_level.groupby("transit"):
    fig.add_scatter(x=data.frequency, y=data.power_average(["segment", "sensor"]).source_level)
fig.show()

In [ ]:
all_sensors.sel(sensor="Colmar 1")

In [ ]:
snr = ship_level.power_average(["segment", "transit", "sensor"]).snr
go.Figure(go.Scatter(x=snr.frequency, y=snr)).update_xaxes(type="log")

In [ ]:
# meets_snr = ship_level.meets_snr_threshold(3).mean(["transit", "segment", "sensor"]) * 100
meets_snr = ship_level.meets_snr_threshold(3).mean(["sensor", "segment", "transit"]) * 100
# meets_snr = ship_level.power_average(["sensor", "segment"]).meets_snr_threshold(3).mean("transit") * 100
go.Figure(go.Scatter(x=meets_snr.frequency, y=meets_snr)).update_xaxes(type="log")

In [ ]:
spread = ship_level.source_level.std(["sensor", "segment", "transit"])
go.Figure(go.Scatter(x=spread.frequency, y=spread)).update_xaxes(type="log")

In [ ]:
cpa
segments = transit.aspect_segments(
    reference=recording.sensor,
    angles=[-45, -30, -15, 0, 15, 30, 45],
    segment_min_length=100,
)
display(transit.subwindow(segments, extend=10).time_window)
display(transit.subwindow(segments).time_window)

In [ ]:
transit.subwindow(segments.sel(segment=-15, edge="center").time)

In [ ]:
ship_level = uwacan.analysis.ShipLevel.analyze_transits(
    *fireface_transits,
    filterbank=filterbank,
    propagation_model=propagation_model,
    background_noise=background,
    transit_min_angle=60,
    transit_min_duration=30,
)

In [ ]:
ship_level._data.set_coords("latitude")

In [ ]:
results.mean("segment")

In [ ]:
# transit.track.resample(segment.time.sel(edge="center"))._dataf
transit.track.resample(segments.time.sel(edge="center"))._data

In [ ]:
source_pos = transit.track.resample(segments.time.sel(edge="center"))._data
segments.sel(edge="center").longitude - source_pos.longitude

In [ ]:
transits = fireface_transits[:2]
aspect_angles = [-45, -30, -15, 0, 15, 30, 45]
aspect_segment_length = 100
aspect_segment_angle = None
aspect_segment_duration = None

if filterbank is None:
    filterbank = uwacan.analysis.NthDecadeSpectrogram(
        bands_per_decade=10,
        lower_bound=20,
        upper_bound=20_000,
        frame_step=1
    )

if background_noise is None:
    def background_noise(received_power, **kwargs):
        return received_power

if propagation_model is None:
    propagation_model = uwacan.propagation.MlogR(m=20)

if isinstance(propagation_model, uwacan.propagation.PropagationModel):
    propagation_model = propagation_model.compensate_propagation

results = []
try:
    transit_padding = filterbank.frame_settings["duration"]
except AttributeError:
    transit_padding = 10
for transit in transits:
    segments = transit.track.aspect_segments(
        reference=transit.recording.sensor,
        angles=aspect_angles,
        segment_min_length=aspect_segment_length,
        segment_min_angle=aspect_segment_angle,
        segment_min_duration=aspect_segment_duration,
    )
    transit = transit.subwindow(segments, extend=transit_padding)

    if np.min(np.abs(segments.segment)) == 0:
        cpa_time = segments.sel(segment=0, edge="center")["time"].data
    else:
        cpa_time = transit.track.closest_point(transit.recording.sensor)["time"].data

    time_data = transit.recording.time_data()
    received_power = filterbank(time_data)
    # track = transit.track.resample(received_power.time)
    # source_pos = transit.track.resample(segments.time)._data

    segment_powers = []
    for segment_angle, segment in segments.groupby("segment", squeeze=False):
        segment_power = received_power.subwindow(segment).mean("time").data
        segment_power.coords["segment"] = segment_angle
        segment_powers.append(segment_power)
    segment_powers = xr.concat(segment_powers, "segment")
    segment_powers = uwacan.analysis.FrequencyData(segment_powers)
    compensated_power = background_noise(segment_powers)
    track = transit.track.resample(segments.sel(edge="center", drop=True).time)
    source_power = propagation_model(received_power=segment_powers, receiver=transit.recording.sensor, source=track)
    transit_time = (track._data["time"] - cpa_time) / np.timedelta64(1, "s")
    transit_results = xr.Dataset(
        data_vars=dict(
            source_power=source_power.data,
            received_power=compensated_power.data,
            latitude=track.latitude,
            longitude=track.longitude,
            transit_time=transit_time,
        ),
        coords=dict(
            # segment=segment,
            # direction=direction,
        )
    )
    results.append(transit_results)
results = xr.concat(results, "transit")


In [ ]:
results

In [ ]:
ship_level

In [ ]:
segments.time.max()

In [ ]:
received_power.time

In [ ]:
transit_results

In [ ]:
transit_results = xr.Dataset(
    data_vars=dict(
        source_power=source_power.data,
        received_power=compensated_power.data,
        latitude=track.latitude,
        longitude=track.longitude,
        transit_time=transit_time,
    ),
    coords=dict(
        # segment=segment_angle,
        # direction=direction,
    )
)

In [ ]:
transit_results

In [ ]:
transits = fireface_transits
aspect_angles = [-45, -30, -15, 0, 15, 30, 45]
aspect_segment_length = 100
aspect_segment_angle = None
aspect_segment_duration = None

if filterbank is None:
    filterbank = uwacan.analysis.NthDecadeSpectrogram(
        bands_per_decade=10,
        lower_bound=20,
        upper_bound=20_000,
        frame_step=1
    )

if background_noise is None:
    def background_noise(received_power, **kwargs):
        return received_power

if propagation_model is None:
    propagation_model = uwacan.propagation.MlogR(m=20)

if isinstance(propagation_model, uwacan.propagation.PropagationModel):
    propagation_model = propagation_model.compensate_propagation

results = []
try:
    transit_padding = filterbank.frame_settings["duration"]
except AttributeError:
    transit_padding = 10
for transit in transits:
    segments = transit.track.aspect_segments(
        reference=transit.recording.sensor,
        angles=aspect_angles,
        segment_min_length=aspect_segment_length,
        segment_min_angle=aspect_segment_angle,
        segment_min_duration=aspect_segment_duration,
    )
    transit = transit.subwindow(segments, extend=transit_padding)

    if np.min(np.abs(segments.segment)) == 0:
        cpa_time = segments.sel(segment=0, edge="center")["time"].data
    else:
        cpa_time = transit.track.closest_point(transit.recording.sensor)["time"].data

    time_data = transit.recording.time_data()
    received_power = filterbank(time_data)
    track = transit.track.resample(received_power.time)
    source_pos = transit.track.resample(segments.time)._data

    received_powers = []
    for segment_angle, segment in segments.groupby("segment"):
        received_segment = received_power.subwindow(segment).mean("time")
        compensated_segment = background_noise(received_segment)
        # segment_time = segment.time.sel(edge="center")
        # source_pos = track.subwindow(segment.time.sel(edge="center"))
        source_power = propagation_model(
            received_power=compensated_segment,
            receiver=transit.recording.sensor,
            source=source_pos.sel(segment=segment_angle),
        )
        transit_time = (segment.time - cpa_time) / np.timedelta64(1, "s")  # The time of this segment relative to cpa
        segment_results = xr.Dataset(
            data_vars=dict(
                source_power=source_power.data,
                received_power=compensated_segment.data,
                latitude=segment.latitude,
                longitude=segment.longitude,
                transit_time=transit_time,
            ),
            coords=dict(
                segment=segment_angle,
            )
        )
        if hasattr(compensated_segment, "snr"):
            segment_results["snr"] = compensated_segment.snr.data
        transit_results.append(segment_results)
    transit_results = xr.concat(transit_results, "segment")
    results.append(transit_results)
results = xr.concat(results, "transit")
results.coords["transit"] = np.arange(results.sizes["transit"]) + 1
bv_source = uwacan.analysis.ShipLevel(results)

In [ ]:
(segments.time - cpa_time) / np.timedelta64(1, "s")